# Imports


In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns


2022-03-08 20:27:53.969095: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-08 20:27:53.969116: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
from stockanalysis.data import *
from stockanalysis.encoder import *

from stockanalysis.utils import *


from stockanalysis.param import *
from stockanalysis.predict import *



In [3]:
import datetime
import joblib


In [5]:
ticker = "INFY.NS"


#Get Data


start_date = (datetime.datetime.now() - datetime.timedelta(days=5 * 365)).strftime("%Y-%m-%d")
end_date = datetime.datetime.now().strftime("%Y-%m-%d")
cleaned_data = get_technical(symbol=ticker, start=start_date,
                                 end=end_date)



[*********************100%***********************]  1 of 1 completed


In [7]:
model = joblib.load(f'../{ticker}.joblib')

2022-03-08 20:29:36.390403: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-08 20:29:36.390484: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-08 20:29:36.390545: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (thierry-Aspire-E5-575G): /proc/driver/nvidia/version does not exist
2022-03-08 20:29:36.392021: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Evaluating the test set


In [4]:
def evaluate(self, x_test, y_test, model):
    '''returns the value of the RMSE'''
    y_pred = model.predict(x_test)

    if TYPE_Y == 'log':
        y_test = np.exp(y_test)
        y_pred = np.exp(y_pred)
        residuos = y_test - y_pred

    #Metrics
    rmse = (residuos ** 2).mean(axis = 0) ** 0.5
    mpe = abs(y_pred / y_test).mean(axis = 0)
    mae = abs(y_pred - y_test).mean(axis = 0)
    df_pred = pd.DataFrame(y_pred).pct_change()
    df_test = pd.DataFrame(y_test).pct_change()
    df_sum_total_errors_pct = (df_pred - df_test).sum(axis = 0) * 100
    df_mean_total_errors_pct = (df_pred - df_test).mean(axis = 0) * 100
    df_max_total_errors_pct = (df_pred - df_test).max(axis = 0) * 100
    df_min_total_errors_pct = (df_pred - df_test).min(axis = 0) * 100


    #Ploting
    fig = plt.figure(figsize= (15,30))
    for i in range(1,Y_LEN):
        if i == 1:
            count = i
        else:
            count +=1
        #Original scale plot
        axs = fig.add_subplot(Y_LEN,2,count)
        axs.set_title = f'{i} days in the future'
        axs.plot(y_test[:,i-1], label= 'y_test')
        axs.plot(y_pred[:,i-1], label= 'y_pred')
        plt.legend()
        #Pct_change plot
        count +=1
        axs = fig.add_subplot(Y_LEN,2,count)
        axs.plot(df_pred.to_numpy()[:,i-1], label= 'y_test')
        axs.plot(df_test.to_numpy()[:,i-1], label= 'y_pred')
        plt.legend()


    plt.show()


    return mpe



# Future Prediction


In [30]:
def prediction(ticker, start, end, model):
   
    print(ticker, start, end)
    df = get_technical(symbol=ticker, start=start, end=end)
    print(df.tail(5))
    cleaned_data = clean_data(df.drop(columns=['Date']))[COLUMNS]
    
    #Creating Sequence to pass to the prediction
    X = cleaned_data.to_numpy()[-(SEQUENCE_SIZE+1):, :]
    
    y = cleaned_data['Close'].to_numpy()[-(SEQUENCE_SIZE+1):]
    print(X.shape, y.shape)
    X, y = split_timeseries(X, y, sequence_size=SEQUENCE_SIZE)
    print(X.shape, y.shape)
    
    results = model.predict(X)
    
    pred_values = np.exp(results[0])
    pct_change_predictions = pd.Series(np.exp(results[0])).pct_change() * 100
    cumsum_pctchange_predictions = pct_change_predictions.cumsum()
    
    
    
    return {"prediction": pred_values, 
            'actual_prices': y, 
            'pct_change_predictions': pct_change_predictions,
            'cumsum_pctchange_predictions': cumsum_pctchange_predictions
           }



In [31]:
import datetime

end_date = datetime.datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.datetime.now() - datetime.timedelta(days=5 * 365)).strftime("%Y-%m-%d")


pred = prediction(ticker=ticker, start=start_date, end=end_date, model=model)




INFY.NS 2017-03-09 2022-03-08
[*********************100%***********************]  1 of 1 completed
           Date         Open         High          Low        Close    Volume  \
1018 2022-03-02  1710.000000  1714.400024  1691.000000  1702.800049   6771489   
1019 2022-03-03  1720.199951  1733.900024  1705.550049  1720.849976   7344147   
1020 2022-03-04  1695.250000  1736.000000  1681.000000  1723.300049   9074499   
1021 2022-03-07  1697.000000  1751.000000  1696.000000  1739.849976   9742588   
1022 2022-03-08  1718.449951  1800.000000  1718.449951  1778.199951  10983354   

            ema12        ema21        ema26        ema34  ...        hma99  \
1018  1716.875788  1730.492937  1737.567362  1746.452948  ...  1757.450557   
1019  1717.487202  1729.616305  1736.329037  1744.989921  ...  1751.664682   
1020  1718.381486  1729.042099  1735.363927  1743.750500  ...  1746.128231   
1021  1721.684330  1730.024634  1735.696227  1743.527613  ...  1741.040611   
1022  1730.379041  1734.

In [32]:
pred

{'prediction': array([1666.5277, 1659.7291, 1664.0431, 1680.2603, 1687.2653, 1683.0724,
        1699.3655, 1701.1605, 1686.978 , 1703.464 , 1710.2603, 1719.3658,
        1717.976 , 1706.4878, 1714.5486], dtype=float32),
 'actual_prices': array([1778.19995117]),
 'pct_change_predictions': 0          NaN
 1    -0.407946
 2     0.259924
 3     0.974560
 4     0.416899
 5    -0.248504
 6     0.968051
 7     0.105631
 8    -0.833696
 9     0.977254
 10    0.398970
 11    0.532413
 12   -0.080836
 13   -0.668705
 14    0.472367
 dtype: float32,
 'cumsum_pctchange_predictions': 0          NaN
 1    -0.407946
 2    -0.148022
 3     0.826538
 4     1.243436
 5     0.994933
 6     1.962984
 7     2.068615
 8     1.234919
 9     2.212173
 10    2.611142
 11    3.143555
 12    3.062719
 13    2.394015
 14    2.866381
 dtype: float32}